In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from datetime import datetime

from feature_builder import process_dataset


In [2]:
x_train = pd.read_csv('train.csv')
y_train=x_train[['id','target']]

In [ ]:
X_train = process_dataset(train)

In [ ]:
#Parametros a usar de manera aleatoria
params = {

        'learning_rate': [ 0.1, 0.2, 0.3, 0.35],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 6],
        'n_estimators': [ 600, 800, 1000, 1200, 1400, 1600]
    }

#Solo sirve para calcular el tiempo que lleva el random search
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
def random_search_func(x, y):

    folds = 7
    param_comb = 570   #Tarda 30 mins. en usar 40 combinaciones
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
    
    xgb = XGBClassifier(objective='binary:logistic',
                    silent=True)

    random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='f1', n_jobs=4, cv=skf.split(x,y), verbose=3, random_state=1001 )

    
    start_time = timer(None) 
    random_search.fit(x, y)
    timer(start_time)   

    print('\n Mejor f1 score con %d-folds y %d combinaciones de parametros:' % (folds, param_comb))
    print(random_search.best_score_)
    print('\n Best hyperparameters:')
    print(random_search.best_params_)

In [ ]:
v = TfidfVectorizer(stop_words = 'english', analyzer = 'word')
x_tfidf = v.fit_transform(x_train['text'])


xtrain, xvalid, ytrain, yvalid=train_test_split(x_tfidf,y_train,test_size=0.2,random_state=1)

In [ ]:
random_search_func(xtrain,ytrain['target'])

In [ ]:

clf = XGBClassifier(max_depth=3, n_estimators=1400, colsample_bytree=0.6,
                        subsample=1, nthread=10, learning_rate=0.3)

xgb_train=clf.fit(xtrain, ytrain['target'])

prediction = clf.predict(xvalid)

In [ ]:
accuracy_score(yvalid['target'], prediction)

In [ ]:
f1_score(prediction, yvalid['target'])

Utilizo word2vec y los features de feature_builder

In [3]:
x_train_features = process_dataset(x_train)

Percentage of words covered in the embeddings = 0.6336399642263958


In [4]:
x_train_feat, x_test_feat, y_train_feat, y_test_feat = train_test_split(x_train_features, y_train, test_size = .33, random_state = 17)

In [5]:
xgbooster = XGBClassifier(max_depth=4, n_estimators=1000, colsample_bytree=1.0,
                        subsample=1, nthread=10, learning_rate=0.2)
xgbooster.fit(x_train_feat, y_train_feat['target'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1.0, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=10, nthread=10, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [6]:
predictions_features = xgbooster.predict(x_test_feat)
accuracy_score(y_test_feat['target'], predictions_features)

0.7978511738957421

In [7]:
f1_score(y_test_feat['target'], predictions_features)

0.7550626808100288